<a href="https://colab.research.google.com/github/lis55/stockPrediction/blob/main/stockPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
data = pd.read_csv('stock_prices.csv')
data

,Date,Open,High,Low,Close,Volume
0,2013-01-02,7.000000,7.090000,6.942000,7.072000,5974000
1,2013-01-03,7.036000,7.090000,6.950000,6.954000,3710000
2,2013-01-04,6.960000,6.960000,6.784000,6.880000,3370000
3,2013-01-07,6.960000,6.960000,6.780000,6.868000,2210000
4,2013-01-08,6.900000,6.900000,6.622000,6.736000,6420000
...,...,...,...,...,...,...
2133,2021-06-23,632.000000,657.200012,630.039978,656.570007,31099200
2134,2021-06-24,674.989990,697.619995,667.609985,679.820007,45982400
2135,2021-06-25,689.580017,693.809998,668.700012,671.869995,32496700
2136,2021-06-28,671.640015,694.700012,670.320007,688.719971,21628200


### We will focus on predicting the "close" variable

In [14]:
data_array = data[['Open','High','Low','Close','Volume']]
close = data_array[]
